# Ranch Demo Notebook

This notebook has examples of how to make Ranch calls to run through the complete network building pipeline to develop standard roadway and transit network from scratch

In [1]:
import os
import pickle
import pandas as pd
import geopandas as gpd
from pyproj import CRS
import json
import sys

import ranch
from ranch import sharedstreets
from ranch import Roadway
from ranch import Transit
from ranch import Parameters
from ranch.utils import link_df_to_geojson, point_df_to_geojson
from ranch.logger import RanchLogger

In [2]:
%load_ext autoreload
%autoreload 2

# Remote I/O

User to update, does not necessarily be the ranch directory

In [3]:
# project directory
root_dir = os.path.join("D:/sanjoaquin")

external_dir = os.path.join(root_dir, "data", "external")
interim_dir = os.path.join(root_dir, "data", "interim")

# software directory
ranch_dir = os.path.join("D:/github/Ranch")

In [4]:
parameters = Parameters(ranch_base_dir = ranch_dir)

2022-01-04 23:06:15, INFO: Lasso base directory set as: D:/github/Ranch


## Step 1 - SharedStreets Extraction

Extracts sharedstreets representation of OSM links.
The input for this step is the polygon boundary file for the area.
The output for this step is geojson files from SharedStreets API.

In [6]:
# the polygon file for the area

input_polygon_file = os.path.join(
    external_dir,
    "sharedstreets_extract", 
    "sanjoaquin.shp"
)

In [7]:
# the folder where SharedStreets extractions live

shst_extract_dir = os.path.join(external_dir, "sharedstreets_extract")

In [8]:
# the SharedStreets extraction file is saved to the 'output_dir' argument

ranch.run_shst_extraction(
    input_polygon_file = input_polygon_file,
    output_dir = shst_extract_dir
)

2022-01-04 15:43:38, INFO: Exporting boundry file D:/sanjoaquin\data\external\sharedstreets_extract\boundary.0.geojson
2022-01-04 15:43:38, INFO: extracting for polygon 0


## Step 2 - OSMNX Extraction

Extracts complete OSM attributes using OSMNX.
The input for this step is the polygon boundary file for the area.
The output for this step is geojson files from OSMNX.

In [8]:
# the folder where OSMNX extractions live

osm_extract_dir = os.path.join(external_dir, "osmnx_extract")

In [10]:
# the OSMNX extraction file is saved to the 'output_dir' argument

ranch.run_osmnx_extraction(
    input_polygon_file = input_polygon_file,
    output_dir = os.path.join(external_dir, "osmnx_extract")
)

## Step 3 - Consolidate SharedStreets and OSMNX extractions

consolidates raw roadway data and create initial roadway networks

In [9]:
# build and returns roaday network object from extraction files

roadway_network = Roadway.create_roadway_network_from_extracts(
    shst_extract_dir = shst_extract_dir,
    osm_extract_dir = osm_extract_dir,
    parameters = parameters,
)

2022-01-04 23:06:48, INFO: Reading sharedstreets data
2022-01-04 23:06:48, INFO: ----------start reading shst extraction data-------------
2022-01-04 23:06:48, INFO: reading shst extraction data : D:/sanjoaquin\data\external\sharedstreets_extract\extract.boundary.0.out.geojson
2022-01-04 23:07:08, INFO: ----------finished reading shst extraction data-------------
2022-01-04 23:07:08, INFO: Removing duplicates in shst extraction data
2022-01-04 23:07:08, INFO: ...before removing duplicates, shst extraction has 99961 geometries.
2022-01-04 23:07:08, INFO: ...after removing duplicates, shst extraction has 99961 geometries.
2022-01-04 23:07:08, INFO: Reading osmnx data
2022-01-04 23:08:44, INFO: Extracting corresponding osm ways for every shst geometry
2022-01-04 23:10:36, INFO: shst extraction has 99961 geometries
2022-01-04 23:10:36, INFO: shst extraction has 107067 osm links
2022-01-04 23:10:36, INFO: ---joining osm shst with osmnx data---
2022-01-04 23:10:36, INFO: shst extraction has 

In [10]:
RanchLogger.info("Initial network has {} links".format(roadway_network.links_df.shape[0]))
RanchLogger.info("Initial network has {} nodes".format(roadway_network.nodes_df.shape[0]))
RanchLogger.info("Initial network has {} shapes".format(roadway_network.shapes_df.shape[0]))

2022-01-04 23:12:46, INFO: Initial network has 192247 links
2022-01-04 23:12:46, INFO: Initial network has 73049 nodes
2022-01-04 23:12:46, INFO: Initial network has 99961 shapes


### Optional: If the user wants to write out the roadway network in standard format, they can do the following, otherwise no need to write out

In [11]:
RanchLogger.info("write out shape geojson")

shape_prop = ['id', 'fromIntersectionId', 'toIntersectionId', 'forwardReferenceId', 'backReferenceId']
shape_geojson = link_df_to_geojson(
    roadway_network.shapes_df, 
    shape_prop
)

with open(os.path.join(interim_dir,"step3_shapes.geojson"), "w") as f:
    json.dump(shape_geojson, f)

RanchLogger.info("write out node geojson")

node_prop = roadway_network.nodes_df.drop('geometry', axis = 1).columns.tolist()
node_geojson = point_df_to_geojson(
    roadway_network.nodes_df, 
    node_prop
)

with open(os.path.join(interim_dir,"step3_nodes.geojson"), "w") as f:
    json.dump(node_geojson, f)

RanchLogger.info("write out link json")

link_prop = roadway_network.links_df.drop(
    ['geometry'], 
    axis = 1
).columns.tolist()

out = roadway_network.links_df[link_prop].to_json(orient = "records")

with open(os.path.join(interim_dir,"step3_links.json"), 'w') as f:
    f.write(out)
    
# the standard format for links are links.json, without the geometry
# here writing out links in links.geojson in case the user wants to visualize links

RanchLogger.info("write out link geojson")

link_prop = roadway_network.links_df.drop('geometry', axis = 1).columns.tolist()
link_geojson = link_df_to_geojson(
    roadway_network.links_df, 
    link_prop
)

with open(os.path.join(interim_dir,"step3_links.geojson"), "w") as f:
    json.dump(link_geojson, f)

2022-01-04 23:12:51, INFO: write out shape geojson
2022-01-04 23:13:09, INFO: write out node geojson
2022-01-04 23:13:21, INFO: write out link json
2022-01-04 23:13:25, INFO: write out link geojson


### Optional: it's more efficient if the user writes out the intermediate network object in pickle

In [12]:
working_network_filename = os.path.join(interim_dir,"step3_network.pickle")
pickle.dump(roadway_network, open(working_network_filename, 'wb'))

## Step 4 - Third-Party Roadway Data Conflation

Conflates e.g. county network with roadway network using SharedStreets.
The input for this step is the 3rd-party geodatabases to conflate.
The output of this step is the SharedStreets conflation outputs in geojson.

In [17]:
# define the input 3rd-party file, either shapefile or geojson

input_network_file = os.path.join(external_dir, "sjmodel", "Network", "2015","TCM_MASTER_2A22_102717.shp")

In [18]:
# if the CRS is undefined in the input network file, user needs to specify:

input_crs = CRS("ESRI:102643")

In [19]:
# the output is saved to the 'output_dir' argument
# user can specify the match option (reference here) 
# by 'custom_match_option'
# if not specified, use the default
# match option would vary by the input network

ranch.run_shst_match(
    input_network_file = input_network_file,
    input_crs = input_crs,
    output_dir = os.path.join(external_dir, "sjmodel", "shst_match"),
    custom_match_option = '--tile-hierarchy=8 --search-radius=50 --snap-intersections'
)

2022-01-04 16:18:56, INFO: input network D:/sanjoaquin\data\external\sjmodel\Network\2015\TCM_MASTER_2A22_102717.shp has crs : None
2022-01-04 16:18:58, INFO: Input network for shst match does not have unique IDs, generating unique IDs
2022-01-04 16:18:58, INFO: Generated 49910 unique IDs for 49910 links in the input network
2022-01-04 16:18:58, INFO: Exporting shst match input - ID-ed geometry file D:/sanjoaquin\data\external\sjmodel\shst_match\TCM_MASTER_2A22_102717.geojson
2022-01-04 16:19:02, INFO: Exporting ID-ed network file D:/sanjoaquin\data\external\sjmodel\shst_match\TCM_MASTER_2A22_102717.full.geojson


## Step 5 - Tidy Up Roadway

Identifies drive dead-ends, cul-de-secs, numbering links and nodes, etc.
The input for this step is the polygon file with subregion identifier, e.g. county.
This step labels each link and node with the county name, and assign model IDs for links and nodes.

In [13]:
# this is the input county polygon file with county names

county_boundary_file = os.path.join(
    external_dir,
    "cb_2018_us_county_500k", 
    "san_joaquin.shp"
)

# specify the column to look for county name

county_variable_name = 'NAME'

In [14]:
roadway_network.tidy_roadway(
    county_boundary_file = county_boundary_file,
    county_variable_name = county_variable_name,
    create_node_link_id = False
)

2022-01-04 23:15:22, INFO: Starting Step 5 Tidy Roadway
2022-01-04 23:15:22, INFO: Joining network with county boundary file for ['San Joaquin'] county
2022-01-04 23:16:01, INFO: Dropping links and nodes that are outside of ['San Joaquin'] county
2022-01-04 23:16:03, INFO: Droppping circular links
2022-01-04 23:16:03, INFO: Flagging dead-end streets for drive network
2022-01-04 23:16:04, INFO: Making dead-end streets drive_access = 0
2022-01-04 23:16:09, INFO: Dropping alternative links between same AB nodes


## Step 6 - Build Transit Network from GTFS

Build standard transit network from GTFS

In [15]:
# the directory where all gtfs feeds are located
# e.g. gtfs_dir/sjrtd_2015_0127/routes.txt
# e.g. gtfs_dir/bart_2015_0127/routes.txt

gtfs_dir = os.path.join(external_dir, "gtfs", "2015")

In [16]:
# read gtfs into transit object

transit_network = Transit.load_all_gtfs_feeds(
    gtfs_dir = gtfs_dir,
    roadway_network= roadway_network,
    parameters=parameters
)

2022-01-04 23:17:24, INFO: Excluding weekend-only services
2022-01-04 23:17:24, INFO: Read and get representative transit feed from: D:/sanjoaquin\data\external\gtfs\2015\sjrtd_2015_0127
2022-01-04 23:17:24, INFO: Lasso base directory set as: D:/github/Ranch


In [18]:
# main activities - routing buses, creating rails

transit_network.build_standard_transit_network(
    multithread_shst_match = True,
    multithread_shortest_path = False
)

2022-01-04 23:17:47, INFO: Getting representative trip for each route by time of day and direction...
2022-01-04 23:17:48, INFO: Snapping gtfs stops to roadway node...
2022-01-04 23:17:55, INFO: Route bus trips using shortest path
2022-01-04 23:17:55, INFO: Routing bus on roadway network from start to end with osmnx...
2022-01-04 23:17:55, INFO: Setting good link dictionary
2022-01-04 23:18:08, INFO: 	Routing agency sjrtd_2015_0127, trip 202920
2022-01-04 23:18:24, INFO: 	Routing agency sjrtd_2015_0127, trip 202370
2022-01-04 23:18:39, INFO: 	Routing agency sjrtd_2015_0127, trip 202358
2022-01-04 23:18:56, INFO: 	Routing agency sjrtd_2015_0127, trip 202906
2022-01-04 23:19:11, INFO: 	Routing agency sjrtd_2015_0127, trip 202923
2022-01-04 23:19:28, INFO: 	Routing agency sjrtd_2015_0127, trip 202427
2022-01-04 23:19:43, INFO: 	Routing agency sjrtd_2015_0127, trip 202954
2022-01-04 23:20:00, INFO: 	Routing agency sjrtd_2015_0127, trip 201399
2022-01-04 23:20:14, INFO: 	Routing agency sjrt

2022-01-04 23:47:00, INFO: 	Routing agency sjrtd_2015_0127, trip 201722
2022-01-04 23:47:17, INFO: 	Routing agency sjrtd_2015_0127, trip 201582
2022-01-04 23:47:31, INFO: 	Routing agency sjrtd_2015_0127, trip 201692
2022-01-04 23:47:48, INFO: 	Routing agency sjrtd_2015_0127, trip 202831
2022-01-04 23:48:04, INFO: 	Routing agency sjrtd_2015_0127, trip 202025
2022-01-04 23:48:21, INFO: 	Routing agency sjrtd_2015_0127, trip 202839
2022-01-04 23:48:35, INFO: 	Routing agency sjrtd_2015_0127, trip 202834
2022-01-04 23:48:52, INFO: 	Routing agency sjrtd_2015_0127, trip 202828
2022-01-04 23:49:07, INFO: 	Routing agency sjrtd_2015_0127, trip 202908
2022-01-04 23:49:24, INFO: 	Routing agency sjrtd_2015_0127, trip 201572
2022-01-04 23:49:38, INFO: 	Routing agency sjrtd_2015_0127, trip 202062
2022-01-04 23:50:58, INFO: Routing bus on roadway network from stop to stop with osmnx...
2022-01-04 23:51:13, INFO: 	Routing agency ['sjrtd_2015_0127'], trip ['202920'], between stops
2022-01-04 23:51:30, IN

2022-01-05 00:11:27, INFO: 	Routing agency ['sjrtd_2015_0127'], trip ['202813'], between stops
2022-01-05 00:11:45, INFO: 	Routing agency ['sjrtd_2015_0127'], trip ['202819'], between stops
2022-01-05 00:12:00, INFO: 	Routing agency ['sjrtd_2015_0127'], trip ['202668'], between stops
2022-01-05 00:12:17, INFO: 	Routing agency ['sjrtd_2015_0127'], trip ['201558'], between stops
2022-01-05 00:12:32, INFO: 	Routing agency ['sjrtd_2015_0127'], trip ['201543'], between stops
2022-01-05 00:12:50, INFO: 	Routing agency ['sjrtd_2015_0127'], trip ['202690'], between stops
2022-01-05 00:13:05, INFO: 	Routing agency ['sjrtd_2015_0127'], trip ['202254'], between stops
2022-01-05 00:13:22, INFO: 	Routing agency ['sjrtd_2015_0127'], trip ['201576'], between stops
2022-01-05 00:13:36, INFO: 	Routing agency ['sjrtd_2015_0127'], trip ['201485'], between stops
2022-01-05 00:13:54, INFO: 	Routing agency ['sjrtd_2015_0127'], trip ['202767'], between stops
2022-01-05 00:14:08, INFO: 	Routing agency ['sjrtd

2022-01-05 00:36:43, INFO: reading shst extraction data : D:/sanjoaquin\data\external\gtfs\2015\gtfs_shape_for_shst_match\match.lines_from_shapes_sjrtd_2015_0127_26269.matched.geojson
2022-01-05 00:36:43, INFO: reading shst extraction data : D:/sanjoaquin\data\external\gtfs\2015\gtfs_shape_for_shst_match\match.lines_from_shapes_sjrtd_2015_0127_26322.matched.geojson
2022-01-05 00:36:43, INFO: reading shst extraction data : D:/sanjoaquin\data\external\gtfs\2015\gtfs_shape_for_shst_match\match.lines_from_shapes_sjrtd_2015_0127_26323.matched.geojson
2022-01-05 00:36:43, INFO: ----------finished reading shst extraction data-------------
2022-01-05 00:36:44, INFO: representative trips include 120 bus shapes, 252 bus trips
2022-01-05 00:36:44, INFO: shst matched 24 bus shapes, 47 bus trips
2022-01-05 00:36:44, INFO: shortest path method matched 96 bus shapes, 205 bus trips
2022-01-05 00:36:44, INFO: Updating stop node matching
2022-01-05 00:37:28, INFO: representative trips include 0 rail/fer

In [19]:
# write out

transit_network.write_standard_transit(
    path = interim_dir
)

## Step 7 - Build Centroid Connectors

builds centroid connectors from TAZ, MAZ

In [ ]:
taz_polygon_file = os.path.join(
    external_dir,
    "taz",
    "SJ_TAZ_Aug2011.shp"
)

In [ ]:
roadway_network.build_centroid_connectors(
    build_taz_drive = True,
    build_taz_active_modes = True,
    input_taz_polygon_file = taz_polygon_file
)

## Step 8 - Write out Standard Format

In [20]:
roadway_network.standard_format(
    county_boundary_file = os.path.join(root_dir,"data", "external","cb_2018_us_county_500k", "san_joaquin.shp"),
    county_variable_name = 'NAME'
)

2022-01-05 00:45:36, INFO: Starting Step 8 creating roadway standard format


In [21]:
RanchLogger.info("write out shape geojson")

shape_prop = ['id', 'fromIntersectionId', 'toIntersectionId', 'forwardReferenceId', 'backReferenceId']
shape_geojson = link_df_to_geojson(
    roadway_network.shapes_df, 
    shape_prop
)

with open(os.path.join(interim_dir,"step8_shapes.geojson"), "w") as f:
    json.dump(shape_geojson, f)

RanchLogger.info("write out node geojson")

node_prop = roadway_network.nodes_df.drop('geometry', axis = 1).columns.tolist()
node_geojson = point_df_to_geojson(
    roadway_network.nodes_df, 
    node_prop
)

with open(os.path.join(interim_dir,"step8_nodes.geojson"), "w") as f:
    json.dump(node_geojson, f)

RanchLogger.info("write out link json")

link_prop = roadway_network.links_df.drop(
    ['geometry'], 
    axis = 1
).columns.tolist()

out = roadway_network.links_df[link_prop].to_json(orient = "records")

with open(os.path.join(interim_dir,"step8_links.json"), 'w') as f:
    f.write(out)
    
# the standard format for links are links.json, without the geometry
# here writing out links in links.geojson in case the user wants to visualize links

RanchLogger.info("write out link geojson")

link_prop = roadway_network.links_df.drop('geometry', axis = 1).columns.tolist()
link_geojson = link_df_to_geojson(
    roadway_network.links_df, 
    link_prop
)

with open(os.path.join(interim_dir,"step8_links.geojson"), "w") as f:
    json.dump(link_geojson, f)

2022-01-05 00:45:49, INFO: write out shape geojson
2022-01-05 00:46:03, INFO: write out node geojson
2022-01-05 00:46:12, INFO: write out link json
2022-01-05 00:46:16, INFO: write out link geojson
